# Project 4: **Build a Deep Research System**
Welcome to project 4! For this project, we shift our focus from tool use and agents to *reasoning* models. You will practice state‑of‑the‑art inference‑time scaling methods such as *Chain‑of‑Thought* prompting and *Tree‑of‑Thoughts*, and briefly explore high-levels of training reasoning models using techniques like **STaR**.


Finally, you will put everything together to build a *deep research agent* that can browse the web, reason over what it finds, and give structured answers.

## Learning Objectives  
* Apply common inference‑time scaling methods: **zero‑shot / few‑shot CoT, self‑consistency, sequential decoding, tree‑of‑thoughts**  
* Gain intuition for **training** reasoning‑capable models following **STaR** approach 
* Build a minimal **deep‑research agent** that combines step‑by‑step reasoning with live web search   
* Practice extending deep-search to a multi-agent system 

## Roadmap  
1. Environment setup  
2. Inference‑time scaling  
   2.1 Few‑shot & zero‑shot CoT  
   2.2 Self‑consistency
   2.3 Sequential revisions  
   2.4 Tree‑of‑Thought
3. STaR for training models for reasoning  
4. Deep-research agent  
5. (Optional) Multi-agent deep-research

# 1‑ Environment setup

## 1.1- Conda environment

Before we start coding, you need a reproducible setup. Open a terminal in the same directory as this notebook and run:

```bash
# Create and activate the conda environment
conda env create -f environment.yaml && conda activate deep_research

# Register this environment as a Jupyter kernel
python -m ipykernel install --user --name=deep_research --display-name "deep_research"
```
Once this is done, you can select "deep_research" from the Kernel → Change Kernel menu in Jupyter or VS Code.

## 1.2 Ollama setup

In this project we use the `llama3.2:3b` and `deepseek-r1:8b` models. You can try other smaller or larger reasoning LLMs such as `qwen2.5:3b-instruct` or `phi4-mini` to compare performance. Explore available models here: https://ollama.com/library.

```bash
ollama pull llama3.2:3b
ollama pull deepseek-r1:8b
# Additional small reasoning models to compare
# ollama pull qwen2.5:3b-instruct
# ollama pull phi4-mini

```

`ollama pull` downloads the model so you can run it locally without API calls.

---  
# 2‑ Inference‑time scaling

Inference-time scaling refers to techniques that make an existing model reason better without retraining it. Instead of changing the model’s weights, we achieve reasoning capability by adjusting how we prompt, sample, or aggregate LLM's outputs.

In this section, we’ll explore several inference-time strategies that improve reasoning quality using a non-reasoning base model. You will experiment with and compare methods such as:

- Few-shot Chain-of-Thought (CoT)
- Zero-shot CoT
- Self-consistency
- Sequential revision
- Tree-of-Thoughts (ToT)

### 2.1: Few‑Shot CoT
Few-shot prompting helps a model reason by showing one or multiple examples before asking a new question. By observing the pattern of reasoning and final answers, the model learns how to structure its own reasoning process on the new input.

In this exercise, you will create a prompt that includes a few example Q&A pairs demonstrating step-by-step reasoning. Then, you will feed a new question and see the model’s output.

In [1]:
# Step 1: Write a few examples showing reasoning steps
# Step 2: Write your new question
# Step 3: Concatenate examples + new question into a single prompt
# Step 4: Call your Ollama or OpenAI client to get a response from llama3.2:3b # e.g., client.chat.completions.create(...)
# Step 5: Print the final answer

from openai import OpenAI

# Initialize client pointing to Ollama running on the host
client = OpenAI(
    api_key="ollama",
    base_url="http://host.docker.internal:11434/v1"
)

MODEL = "llama3.2:3b"

# Few-shot examples with explicit reasoning steps
few_shot_examples = """
Q: A farmer has 3 chickens. Each chicken lays 2 eggs per day. After 4 days, the farmer sells half of the eggs. Then each remaining egg hatches into a chick that will lay 2 eggs per day. How many eggs will the farmer have after 2 more days?

A: Let me work through this step by step:
Step 1: Calculate initial eggs laid
- 3 chickens x 2 eggs/day x 4 days = 24 eggs

Step 2: Calculate eggs after selling half
- Eggs sold: 24 ÷ 2 = 12 eggs
- Eggs remaining: 24 - 12 = 12 eggs

Step 3: Calculate new chickens from hatched eggs
- 12 eggs hatch into 12 new chicks
- Total chickens now: 3 + 12 = 15 chickens

Step 4: Calculate eggs laid by all chickens over 2 days
- 15 chickens x 2 eggs/day x 2 days = 60 eggs

Final answer: 60 eggs

---

Q: A library charges $2 for the first day a book is late, and the fine doubles each additional day. If someone returns a book 5 days late, how much do they owe?

A: Let me solve this step by step:
Step 1: Day 1 fine
- First day: $2

Step 2: Day 2 fine (doubles)
- Second day: $2 x 2 = $4

Step 3: Day 3 fine (doubles again)
- Third day: $4 x 2 = $8

Step 4: Day 4 fine
- Fourth day: $8 x 2 = $16

Step 5: Day 5 fine
- Fifth day: $16 x 2 = $32

Step 6: Total fine
- $2 + $4 + $8 + $16 + $32 = $62

Final answer: $62

---
"""

# A challenging question that requires multi-step reasoning
new_question = """
Q: A swimming pool can be filled by Pipe A in 6 hours and by Pipe B in 4 hours. However, there's a drain that can empty the full pool in 12 hours. If all three (both pipes and the drain) are opened simultaneously when the pool is empty, how long will it take to fill the pool?

A: Let me work through this step by step:
"""

# This problem is harder than simple arithmetic because it involves:
# 
# Converting time-to-complete into rates (1/6, 1/4, 1/12 pools per hour)
# Understanding that the drain works against the pipes
# Combining rates: 1/6 + 1/4 - 1/12 = net rate
# 2/12 + 3/12 - 1/12 = 4/12 = 1/3 pools per hour
# Finding the time to fill the pool at the net rate (3 hours)

# Concatenate examples and question
prompt = few_shot_examples + new_question

# Call the model
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant that solves problems step by step."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7,
    max_tokens=500
)

# Print the response
answer = response.choices[0].message.content
print("Model's reasoning and answer:")
print(answer)

Model's reasoning and answer:
Let's break down the problem of filling the swimming pool. I'll solve it step by step.

Step 1: Calculate rate of each pipe
- Pipe A fills the pool in 6 hours, so its rate is 1/6 of the pool per hour.
- Pipe B fills the pool in 4 hours, so its rate is 1/4 of the pool per hour.

Step 2: Calculate combined rate without drain
- Combined rate = Rate of Pipe A + Rate of Pipe B
= 1/6 + 1/4
To add these fractions, we need a common denominator, which is 12.
= (2/12) + (3/12)
= 5/12

Step 3: Calculate rate of drain
- The drain empties the full pool in 12 hours, so its rate is 1/12 of the pool per hour.

Step 4: Calculate combined rate with drain
- Combined rate = Rate of Pipe A + Rate of Pipe B - Rate of Drain
= 5/12 - 1/12
= 4/12

Step 5: Simplify combined rate
- The fraction 4/12 can be simplified by dividing both numerator and denominator by their greatest common divisor, which is 4.
= 1/3

Step 6: Calculate time to fill the pool
- Time = Rate / Combined Rate
Si

### (Optional) Few-shot CoT on GPT2
GPT-2 is a pre-trained language model without instruction tuning. It continues text rather than answering questions. In this section, you'll try the exact same CoT pattern on GPT-2 and observe what happens. The goal is to test whether few-shot CoT alone can elicit structured reasoning from a non-chat LLM.

In [2]:
import os
import torch
from transformers import pipeline

# Step 1: Load GPT-2 text-generation from huggingface (https://huggingface.co/docs/transformers/en/model_doc/gpt2)
# Step 2: Write 1–2 few-shot reasoning examples (short, explicit steps + final answer in your own unique format)
# Step 3: Append a new test question after the examples to form one prompt string
# Step 4: Generate 1–3 completions with different decoding settings (e.g., greedy vs. top-k)
# Step 5: Print raw outputs; check if steps are followed and if the final answer is correct

# Reuse the same examples and question from the original code
few_shot_examples = """
Q: A farmer has 3 chickens. Each chicken lays 2 eggs per day. After 4 days, the farmer sells half of the eggs. Then each remaining egg hatches into a chick that will lay 2 eggs per day. How many eggs will the farmer have after 2 more days?

A: Let me work through this step by step:
Step 1: Calculate initial eggs laid
- 3 chickens x 2 eggs/day x 4 days = 24 eggs

Step 2: Calculate eggs after selling half
- Eggs sold: 24 ÷ 2 = 12 eggs
- Eggs remaining: 24 - 12 = 12 eggs

Step 3: Calculate new chickens from hatched eggs
- 12 eggs hatch into 12 new chicks
- Total chickens now: 3 + 12 = 15 chickens

Step 4: Calculate eggs laid by all chickens over 2 days
- 15 chickens x 2 eggs/day x 2 days = 60 eggs

Final answer: 60 eggs

---

Q: A library charges $2 for the first day a book is late, and the fine doubles each additional day. If someone returns a book 5 days late, how much do they owe?

A: Let me solve this step by step:
Step 1: Day 1 fine
- First day: $2

Step 2: Day 2 fine (doubles)
- Second day: $2 x 2 = $4

Step 3: Day 3 fine (doubles again)
- Third day: $4 x 2 = $8

Step 4: Day 4 fine
- Fourth day: $8 x 2 = $16

Step 5: Day 5 fine
- Fifth day: $16 x 2 = $32

Step 6: Total fine
- $2 + $4 + $8 + $16 + $32 = $62

Final answer: $62

---
"""

new_question = """
Q: A swimming pool can be filled by Pipe A in 6 hours and by Pipe B in 4 hours. However, there's a drain that can empty the full pool in 12 hours. If all three (both pipes and the drain) are opened simultaneously when the pool is empty, how long will it take to fill the pool?

A: Let me work through this step by step:
"""

print("Loading GPT-2 model...")
# Load GPT-2 text generation pipeline
generator = pipeline(
    "text-generation",
    model="gpt2",
    device=0 if torch.cuda.is_available() else -1
)

print("\n" + "="*80)
print("TEST 1: GPT-2 WITHOUT Chain-of-Thought (Question Only)")
print("="*80)

# Generate without CoT examples
output_no_cot = generator(
    new_question,
    max_new_tokens=200,
    num_return_sequences=1,
    temperature=0.7,
    do_sample=True,
    pad_token_id=generator.tokenizer.eos_token_id
)

print("\nGenerated text (without CoT):")
print(output_no_cot[0]["generated_text"])

print("\n" + "="*80)
print("TEST 2: GPT-2 WITH Chain-of-Thought (Examples + Question)")
print("="*80)

# Generate with CoT examples
prompt_with_cot = few_shot_examples + new_question

output_with_cot = generator(
    prompt_with_cot,
    max_new_tokens=200,
    num_return_sequences=1,
    temperature=0.7,
    do_sample=True,
    pad_token_id=generator.tokenizer.eos_token_id
)

print("\nGenerated text (with CoT):")
# Only print the new generation after the prompt
generated_part = output_with_cot[0]["generated_text"][len(prompt_with_cot):]
print(generated_part)

print("\n" + "="*80)
print("TEST 3: Different Decoding Strategies (with CoT)")
print("="*80)

# Greedy decoding
print("\n--- Greedy Decoding ---")
greedy_output = generator(
    prompt_with_cot,
    max_new_tokens=150,
    num_return_sequences=1,
    do_sample=False,
    pad_token_id=generator.tokenizer.eos_token_id
)
print(greedy_output[0]["generated_text"][len(prompt_with_cot):])

# Top-k sampling
print("\n--- Top-K Sampling (k=50) ---")
topk_output = generator(
    prompt_with_cot,
    max_new_tokens=150,
    num_return_sequences=1,
    do_sample=True,
    top_k=50,
    temperature=0.7,
    pad_token_id=generator.tokenizer.eos_token_id
)
print(topk_output[0]["generated_text"][len(prompt_with_cot):])

# Top-p (nucleus) sampling
print("\n--- Top-P Sampling (p=0.9) ---")
topp_output = generator(
    prompt_with_cot,
    max_new_tokens=150,
    num_return_sequences=1,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
    pad_token_id=generator.tokenizer.eos_token_id
)
print(topp_output[0]["generated_text"][len(prompt_with_cot):])

print("\n" + "="*80)
print("COMPARISON SUMMARY")
print("="*80)
print(f"Without CoT length: {len(output_no_cot[0]['generated_text'])} chars")
print(f"With CoT length: {len(output_with_cot[0]['generated_text'])} chars")
print(f"New generation length: {len(generated_part)} chars")
print("\nNote: GPT-2 is a completion model, not an instruction-tuned chat model.")
print("It may not follow the step-by-step format perfectly, but CoT examples")
print("can still influence the continuation pattern.")

Loading GPT-2 model...


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu



TEST 1: GPT-2 WITHOUT Chain-of-Thought (Question Only)

Generated text (without CoT):

Q: A swimming pool can be filled by Pipe A in 6 hours and by Pipe B in 4 hours. However, there's a drain that can empty the full pool in 12 hours. If all three (both pipes and the drain) are opened simultaneously when the pool is empty, how long will it take to fill the pool?

A: Let me work through this step by step:

Fill a pipe with water

Fill a pipe with water

Fill a pipe with water

Fill a pipe with water

Empty a pipe

Fill a pipe with water

Fill a pipe with water

Fill a pipe with water

Fill a pipe with water

Fill a pipe with water

Fill a pipe with water

Fill a pipe with water

Fill a pipe with water

Fill a pipe with water

Fill a pipe with water

Fill a pipe with water

Fill a pipe with water

Fill a pipe with water

Fill a pipe with water

Fill a pipe with water

Fill a pipe with water

Fill a pipe with water

Fill a pipe with water

Fill a pipe with water

Fill a pipe with water

F

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Generated text (with CoT):

Step 1: Day 1 fine

- First day: $2

- Second day: $2 x 2 = $4

- Third day: $2 x 2 = $16

Step 2: Day 3 fine

- Fourth day: $8 x 2 = $16

Step 3: Day 3 fine (doubles again)

- Fifth day: $16 x 2 = $32

Step 4: Day 4 fine

- Fifth day: $32 x 2 = $32

Step 5: Day 5 fine (doubles again)

- Sixth day: $16 x 2 = $32

Step 6: Day 6 fine

- Seventh day: $32 x 2 = $32

Step 6: Total fine

---

Q: A carpenter can build a house, and yet there are no roof or cabinets, and the house still sits on the ground. How long

TEST 3: Different Decoding Strategies (with CoT)

--- Greedy Decoding ---

Step 1: Pipe A empties the full pool in 12 hours

Step 2: Pipe B empties the full pool in 4 hours

Step 3: Pipe A empties the full pool in 8 hours

Step 4: Pipe B empties the full pool in 12 hours

Final answer: 12 hours

---

Q: A water heater can be used to heat a house. How much heat does it cost to heat a house?

A: Let me solve this step by step:

Step 1: Water heater

- 1 ga

### 2.2: Zero‑Shot Chain‑of‑Thought
Zero-shot CoT encourages the model to reason without examples by adding a short cue such as “Let’s think step by step.” This simple phrase often activates the model’s latent reasoning ability even when no demonstrations are provided. It serves as a baseline to compare with few-shot and other inference-time scaling methods.

In [1]:
from openai import OpenAI

# Step 1: Write the question and a zero-shot CoT cue (e.g., "Let's think step by step.")
# Step 2: Build a single prompt string that includes brief role guidance plus the question
# Step 3: Call your Ollama or OpenAI client to get a response from llama3.2:3b  # e.g., client.chat.completions.create(...)
# Step 4: Print the chain and the final answer

client = OpenAI(
    api_key="ollama",
    base_url="http://host.docker.internal:11434/v1"
)

MODEL = "llama3.2:3b"

prompt = """
A swimming pool can be filled by Pipe A in 6 hours and by Pipe B in 4 hours. However, there's a drain that can empty the full pool in 12 hours. If all three (both pipes and the drain) are opened simultaneously when the pool is empty, how long will it take to fill the pool?

Let's think step by step.
"""

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant that solves problems step by step."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7,
    max_tokens=500
)

answer = response.choices[0].message.content
print("Model's reasoning and answer:")
print(answer)

Model's reasoning and answer:
To solve this problem, we need to find the combined rate at which Pipe A fills the pool, Pipe B fills the pool, and the drain empties the pool. Then, we can calculate the net rate of filling the pool.

Step 1: Find the individual rates of each pipe:

- Pipe A's rate = 1 pool / 6 hours = 1/6 pool per hour
- Pipe B's rate = 1 pool / 4 hours = 1/4 pool per hour

Step 2: Find the drain's rate (negative, because it empties the pool):

- Drain's rate = -1 pool / 12 hours = -1/12 pool per hour (note the negative sign)

Step 3: Calculate the combined rate of filling the pool:

Combined rate = Pipe A's rate + Pipe B's rate + Drain's rate
= 1/6 pool/hour + 1/4 pool/hour - 1/12 pool/hour

To add these fractions, we need a common denominator (12). We'll convert each fraction accordingly:
- 1/6 = 2/12
- 1/4 = 3/12
- -1/12 remains the same

Now, let's calculate the combined rate:

Combined rate = 2/12 + 3/12 - 1/12
= (2+3-1)/12 pool/hour
= 4/12 pool/hour
= 1/3 pool per 

### 2.3 Self‑Consistency
Self-consistency enhances reasoning accuracy by sampling multiple independent reasoning paths for the same question instead of relying on a single deterministic answer. Each run may follow a slightly different logical chain, and the diversity helps correct individual mistakes. After generating several reasoning traces, you then aggregate the final answers using majority voting.

This approach is especially useful when tasks involve multi-step reasoning or arithmetic, where single-path outputs may be incorrect.

In [2]:
from openai import OpenAI
import re, collections

client = OpenAI(api_key = "ollama", base_url = "http://host.docker.internal:11434/v1")
MODEL = "llama3.2:3b"

def cot_answer(question, temperature=1.0):
    # Generate a step-by-step reasoning chain for the given question and extract the final answer.
    prompt = f"{question}\n\nLet's think step by step."
    
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful assistant that solves problems step by step."},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature,
        max_tokens=500
    )
    
    full_response = response.choices[0].message.content
    
    # Extract final answer - look for common patterns
    answer_match = re.search(r'(?:final answer|answer|therefore|thus|so)[:\s]+([^\n.]+)', full_response, re.IGNORECASE)
    if answer_match:
        return answer_match.group(1).strip()
    
    # Fallback: return last line
    return full_response.strip().split('\n')[-1].strip()

def self_consistent(question, n=10):
    # Run multiple reasoning chains and select the most frequent final answer by majority voting.
    answers = []
    
    print(f"\nGenerating {n} reasoning chains...")
    for i in range(n):
        answer = cot_answer(question, temperature=1.0)
        answers.append(answer)
        print(f"  Chain {i+1}: {answer}")
    
    # Count frequency of each answer
    counter = collections.Counter(answers)
    
    # Get most common answer
    winner = counter.most_common(1)[0][0]
    
    return winner, counter


question = "What is the square root of 144?"
winner, counter = self_consistent(question)
print("Votes:", counter)
print("Chosen answer:", winner)


Generating 10 reasoning chains...
  Chain 1: Yes! 12 squared equals 144, so we can confirm
  Chain 2: The square root of 144 is 12
  Chain 3: square root of the number 144 has two factors for both positive and a Negative value
  Chain 4: √144 = ±12
  Chain 5: Is that what you were expecting?
  Chain 6: is:
  Chain 7: And there you have it. The square root of 144 is indeed 12.
  Chain 8: Therefore, the square root of 144 is 12.
  Chain 9: is:
  Chain 10: So, the square root of 144 is indeed 12!
Votes: Counter({'is:': 2, 'Yes! 12 squared equals 144, so we can confirm': 1, 'The square root of 144 is 12': 1, 'square root of the number 144 has two factors for both positive and a Negative value': 1, '√144 = ±12': 1, 'Is that what you were expecting?': 1, 'And there you have it. The square root of 144 is indeed 12.': 1, 'Therefore, the square root of 144 is 12.': 1, 'So, the square root of 144 is indeed 12!': 1})
Chosen answer: is:


### 2.4: Sequential Revision

Sequential revision iteratively improves an answer by generating a first draft, critiquing it, and producing revised drafts that condition on prior answers. Each round should be short and focused, so improvements accumulate without drifting from the question.

In [5]:
from openai import OpenAI
import re

client = OpenAI(api_key="ollama", base_url="http://host.docker.internal:11434/v1")
MODEL = "llama3.2:3b"

def extract_answer(text):
    """Extract numeric answer from text"""
    # Look for "X hours" pattern
    match = re.search(r'(\d+(?:\.\d+)?)\s*hours?', text, re.IGNORECASE)
    if match:
        return match.group(1)
    return "Not found"

def sequential_revision(question: str, max_steps: int = 3) -> str:
    # Generate an initial draft answer, then iteratively refine it by conditioning each revision on the previous one.
    
    # Step 1: Generate initial draft
    print(f"\n{'='*80}")
    print(f"DRAFT 1 (Initial Answer)")
    print(f"{'='*80}")
    
    prompt = f"{question}\n\nProvide a detailed answer with step-by-step reasoning."
    
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful assistant that solves problems step by step."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=500
    )
    
    current_draft = response.choices[0].message.content
    print(current_draft)
    print(f"\nExtracted answer: {extract_answer(current_draft)} hours")
    
    # Step 2: Iteratively revise the draft
    for step in range(2, max_steps + 1):
        print(f"\n{'='*80}")
        print(f"DRAFT {step} (Revision {step-1})")
        print(f"{'='*80}")
        
        revision_prompt = f"""Question: {question}

Previous answer:
{current_draft}

Please review the previous answer and provide an improved version that:
1. Fixes any errors or inaccuracies
2. Adds more clarity and detail
3. Ensures the reasoning is correct

Provide your revised answer:"""
        
        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": "You are a helpful assistant that carefully reviews and improves answers."},
                {"role": "user", "content": revision_prompt}
            ],
            temperature=0.7,
            max_tokens=500
        )
        
        current_draft = response.choices[0].message.content
        print(current_draft)
        # print(f"\nExtracted answer: {extract_answer(current_draft)} hours")
    
    # Step 4: Return the final improved draft
    return current_draft


# Test with the pool filling question
question = """A swimming pool can be filled by Pipe A in 6 hours and by Pipe B in 4 hours. However, there's a drain that can empty the full pool in 12 hours. If all three (both pipes and the drain) are opened simultaneously when the pool is empty, how long will it take to fill the pool?"""

print("="*80)
print("SEQUENTIAL REVISION TEST")
print("="*80)
print("Expected answer: 3 hours")
print("Correct reasoning: 1/6 + 1/4 - 1/12 = 2/12 + 3/12 - 1/12 = 4/12 = 1/3 pool/hour → 3 hours")

final_answer = sequential_revision(question, max_steps=3)

print(f"\n{'='*80}")
print("FINAL ANSWER")
print(f"{'='*80}")
print(final_answer)
# print(f"\nFinal extracted answer: {extract_answer(final_answer)} hours")

print("\n" + "="*80)
print("ANALYSIS: Why Sequential Revision Can Fail")
print("="*80)
print("""
Issues observed:
1. Model may confuse itself through vague critique prompts
2. No external verification or grounding
3. Can "improve" away from correct answer
4. Temperature=0.7 adds randomness to revisions

Better approaches:
- Use self-consistency BEFORE revision (get consensus first)
- Add explicit verification steps (check arithmetic)
- Use lower temperature for revisions (0.1-0.3)
- Stop early if answer is already correct
- Add constraints (e.g., "verify all calculations")
""")

SEQUENTIAL REVISION TEST
Expected answer: 3 hours
Correct reasoning: 1/6 + 1/4 - 1/12 = 2/12 + 3/12 - 1/12 = 4/12 = 1/3 pool/hour → 3 hours

DRAFT 1 (Initial Answer)
To solve this problem, let's break down the process into steps:

**Step 1: Calculate the rate of filling for each pipe and the drain**

 Pipe A fills the pool in 6 hours, so its rate is:
Rate of Pipe A = 1 pool / 6 hours
= 1/6 pool per hour

Pipe B fills the pool in 4 hours, so its rate is:
Rate of Pipe B = 1 pool / 4 hours
= 1/4 pool per hour

The drain empties the pool in 12 hours, so its rate is:
Rate of Drain = -1 pool / 12 hours (negative sign indicates emptying)

**Step 2: Calculate the combined rate**

When all three pipes and the drain are opened simultaneously, their rates add up. To find the combined rate, we need to subtract the rate of the drain from the sum of the rates of Pipe A and Pipe B:
Combined Rate = Rate of Pipe A + Rate of Pipe B - Rate of Drain
= (1/6) + (1/4) - (-1/12)
To add these fractions, we nee

### 2.5 Tree‑of‑Thoughts
Tree-of-Thoughts reframes reasoning as a search process rather than a single forward chain.
Instead of producing one linear sequence of thoughts, the model generates multiple candidate thoughts at each step, evaluates their promise, and then expands only the best few. This allows exploration of different reasoning paths before committing to a final answer, similar to how humans brainstorm, prune, and refine ideas.


In this section, you’ll experiment with two simplified versions of ToT:
1. Word Ladder puzzle solver: a small example where each “thought” is a candidate word transition.
2. Generic ToT search (depth 2, width 2): a minimal logic to expand, evaluate, and select reasoning branches

In [6]:
###### Word Ladder Puzzle ##########

def neighbors(word, vocabulary):
    # Generate all valid one-letter mutations of 'word' that exist in 'vocabulary' and return them.
    result = []
    for i in range(len(word)):
        for c in 'abcdefghijklmnopqrstuvwxyz':
            if c != word[i]:
                candidate = word[:i] + c + word[i+1:]
                if candidate in vocabulary:
                    result.append(candidate)
    return result

def tree_of_thought(start, goal, vocab, max_depth=5, beam_width=4):
    # Search over partial thoughts (paths) using a small beam.
    # Step 1: Initialize the frontier with a single path [start]
    frontier = [[start]]
    
    # Step 2: For each depth, expand each path by one neighbor from 'neighbors'
    for depth in range(max_depth):
        candidates = []
        
        for path in frontier:
            last_word = path[-1]
            
            # Check if we've reached the goal
            if last_word == goal:
                return path
            
            # Expand path with all valid neighbors
            for neighbor in neighbors(last_word, vocab):
                if neighbor not in path:  # Avoid cycles
                    candidates.append(path + [neighbor])
        
        if not candidates:
            return None  # No valid paths found
        
        # Step 3: Score paths by edit distance between last word and 'goal' (smaller is better)
        def edit_distance(w1, w2):
            return sum(c1 != c2 for c1, c2 in zip(w1, w2))
        
        scored_candidates = [(edit_distance(path[-1], goal), path) for path in candidates]
        
        # Step 4: Keep the top 'beam_width' paths (sort by score ascending)
        scored_candidates.sort(key=lambda x: x[0])
        frontier = [path for score, path in scored_candidates[:beam_width]]
    
    # Step 5: Return the best path or None
    return frontier[0] if frontier else None

vocab = {"hit","dot","cog","log","dog","lot","lit","hot"}
print(tree_of_thought("hit", "cog", vocab)) # one candidate solution: ['hit', 'hot', 'dot', 'dog', 'cog']


['hit', 'hot', 'dot', 'dog', 'cog']


In [7]:
###### Generic ToT Search ##########

import re
from openai import OpenAI

client = OpenAI(api_key="ollama", base_url="http://host.docker.internal:11434/v1")
MODEL = "llama3.2:3b"

def propose_thoughts(question, state, k=2):
    # Propose up to k next "thoughts" that extend the current partial solution/state.
    # Steps: build a short prompt with problem + current state; call your client with n=k. Then return a list of stripped strings (≤ k).
    
    if state == "":
        prompt = f"Question: {question}\n\nPropose the first step to solve this problem. Be concise (1-2 sentences)."
    else:
        prompt = f"Question: {question}\n\nCurrent progress:\n{state}\n\nPropose the next step to continue solving this. Be concise (1-2 sentences)."
    
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.8,
        max_tokens=100,
        n=k  # Generate k different thoughts
    )
    
    thoughts = [choice.message.content.strip() for choice in response.choices]
    return thoughts


def score_state(question, state):
    # Score how promising a partial solution is on a 1–10 scale (higher is better).
    # Steps: build a rating prompt; call the model; parse the first integer 1–10;
    
    prompt = f"""Question: {question}

Current partial solution:
{state}

Rate how promising this partial solution is on a scale of 1-10, where:
- 1-3: Poor progress, unlikely to lead to a good solution
- 4-6: Decent progress, some potential
- 7-9: Good progress, likely to lead to a strong solution
- 10: Excellent, nearly complete solution

Respond with ONLY a number from 1-10."""
    
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=10
    )
    
    # Parse the score from response
    text = response.choices[0].message.content.strip()
    match = re.search(r'\b([1-9]|10)\b', text)
    score = int(match.group(1)) if match else 5  # Default to 5 if parsing fails
    
    return score


def tree_of_thoughts(question, depth=2, width=2):
    # Run a tiny ToT search: expand states with propose_thoughts, score with score_state, keep top-k at each depth.
    # Steps: initialize frontier=[("", 0)]; for each depth, expand each state with k=width thoughts; score each; sort by score desc; keep top 'width'; return best state and score.
    
    # Initialize frontier with empty state
    frontier = [("", 0)]  # (state, score)
    
    for d in range(depth):
        print(f"\n{'='*60}")
        print(f"Depth {d+1}/{depth}")
        print(f"{'='*60}")
        
        candidates = []
        
        # Expand each state in frontier
        for state, _ in frontier:
            # Generate width new thoughts from this state
            thoughts = propose_thoughts(question, state, k=width)
            
            for thought in thoughts:
                # Build new state by appending thought
                if state == "":
                    new_state = thought
                else:
                    new_state = f"{state}\n\n{thought}"
                
                # Score the new state
                score = score_state(question, new_state)
                candidates.append((new_state, score))
                
                print(f"\nThought: {thought[:80]}...")
                print(f"Score: {score}/10")
        
        # Keep only top 'width' candidates
        candidates.sort(key=lambda x: x[1], reverse=True)
        frontier = candidates[:width]
        
        print(f"\nTop {width} states kept:")
        for i, (s, sc) in enumerate(frontier, 1):
            print(f"{i}. Score {sc}/10: {s[:100]}...")
    
    # Return the best state and its score
    best_state, best_score = frontier[0]
    return best_state, best_score


question = "Design a plan for a weekend science workshop for 12-year-olds."
solution, score = tree_of_thoughts(question, depth=2, width=2)

print(f"\n{'='*60}")
print("FINAL BEST SOLUTION")
print(f"{'='*60}")
print(f"Score: {score}/10\n")
print(solution)


Depth 1/2

Thought: The first step to designing a plan for a weekend science workshop for 12-year-ol...
Score: 8/10

Top 2 states kept:
1. Score 8/10: The first step to designing a plan for a weekend science workshop for 12-year-olds would be to deter...

Depth 2/2

Thought: The next step would be to identify a target group's needs and abilities by condu...
Score: 8/10

Top 2 states kept:
1. Score 8/10: The first step to designing a plan for a weekend science workshop for 12-year-olds would be to deter...

FINAL BEST SOLUTION
Score: 8/10

The first step to designing a plan for a weekend science workshop for 12-year-olds would be to determine the specific theme or topic(s) of interest that will guide the content and activities, such as space exploration, environmental science, or chemistry experiments.

This step involves deciding what type of hands-on learning experiences, lectures, and discussions will engage participants and align with their interests.

The next step would be to ide

---  
# 3‑ Training Models for Reasoning

### 3.1: CoT Training
Chain-of-Thought (CoT) training conditions the model on explicit rationales during fine-tuning. Instead of teaching the model to output only the final answer, we train on (question, rationale, answer) so the model learns to internalize multi-step reasoning patterns. A practical recipe is STaR (Self-Taught Reasoner), which uses a stronger teacher model to bootstrap rationales that a smaller student can learn from.

For tasks that require multi-hop reasoning, models fine-tuned on rationales often achieve higher accuracy and are more stable at inference time than models trained on direct answers only. 

Training a full language model is beyond the scope of this notebook, but here is the high-level workflow followed by a short pseudocode:
- Collect questions: Prepare a dataset of questions and correct answers.
- Generate rationales: Use a strong LLM to produce step-by-step reasoning ending with the correct answer.
- Filter and clean: Discard incorrect or low-quality rationales.
- Prepare training data: Format triples (question, rationale, answer) for supervised fine-tuning.
- Fine-tune: Fine-tune the LLM on rationales.
- Iterate: Refine prompts, improve data quality, and retrain for stronger reasoning.

In [ ]:
# Pseudocode (STaR loop)
# for round in 1 ... iters:
    # STEP 1: self-generate reasoning (teacher creates rationale + answer)
    # STEP 2: keep only correct, high-quality traces
    # STEP 3: fine-tune student on (question, rationale, answer) data

### 3.2: ORM vs PRM + RL
Training a Reward Model (RM) allows large language models to be improved through reinforcement learning (RL). Instead of fine-tuning directly on examples, we train a separate model that can score or rank model outputs, and use those scores as feedback signals to refine the policy model.

Two main reward modeling approaches are ORM (predicts a scalar reward for the final answer) and PRM (evaluates the reasoning steps instead of just the outcome)



| Approach | Typical loss | When to use |
|-----------|-------------|-------------|
|*Outcome Reward Model* | Predict scalar reward | Easy to collect training data using verifiers |
|*Process Reward Model* | Predict rewards per step | Difficult to collect training data but more accurate |
| *RLHF* | Use RM as reward in **RL** fine‑tuning | Aligns policy with human signals | Aligns model policy with human or synthetic preferences




In [ ]:
# for round = 1 ... iters:
    # STEP 1:  Generate reasoning
        # sample a minibatch of questions
        # policy roll‑out (actions + log‑probs)
    # STEP 2:  Score the trajectory
        # ORM: scalar reward for the final answer / PRM: scalar reward for the thought process
    # STEP 3:  Reinforce the policy (PPO)

---  
# 4‑ A Deep Research Agent

A deep-research agent pairs a reasoning model (e.g., deepseek-r1) with external tools for web search and retrieval. We will follow the ReAct pattern: the model writes short thoughts, decides when to call tools, reads observations, and continues reasoning until it can answer or reaches a step limit.

We now combine a **search tool** with a reasoning model (e.g., `deepseek-r1`) in a multi-step setup. We follow the *ReAct* pattern (reason → tool → observation):

1. The model reasoins and decides to use tools
2. The agent searches and feed condensed snippets back as context
3. Iterate until the model answers or hits a step limit

We use `AgentType.OPENAI_FUNCTIONS`, which hides the loop inside the LangChain agent.

In [9]:
from ddgs import DDGS
from langchain.tools import Tool

def ddg_search(query: str, k: int = 5) -> str:
    # Use DDGS to run a simple web search and return joined snippets.
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=k))
    
    # Extract and join snippets from search results
    snippets = [f"[{r['title']}] {r['body']}" for r in results]
    return "\n\n".join(snippets)

search_tool = Tool(
    name="DuckDuckGo Search",
    func=ddg_search,
    description="Search the public web. Input: a plain English query. Returns: concatenated snippets."
)

In [10]:
from langchain.agents import initialize_agent, AgentType
from langchain_community.chat_models import ChatOllama

#MODEL = "deepseek-r1:8b"
MODEL = "llama3.2:3b"
question = "What are the best resources to learn machine learning in 2025?"

# Step 1: Initialize the reasoning model via ChatOllama
llm = ChatOllama(model=MODEL, base_url="http://host.docker.internal:11434")

# Step 2: Build the agent with tool access (DuckDuckGo Search) and function-calling interface (initialize_agent)
agent = initialize_agent([search_tool], llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Step 3: Ask a query and let the agent search + reason to produce an answer
result = agent.run(question)
print("\n" + "="*80)
print("FINAL ANSWER")
print("="*80)
print(result)

/tmp/ipykernel_1439/3397914316.py:9: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model=MODEL, base_url="http://host.docker.internal:11434")
/tmp/ipykernel_1439/3397914316.py:12: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/do



> Entering new AgentExecutor chain...
Thought: To find the best resources to learn machine learning in 2025, I need to search for relevant information on the web.

Action: DuckDuckGo Search
Action Input: "best machine learning resources 2025"
Observation: [7 Best Machine Learning Courses for 2025 (read this first) – LearnDataSci] If you take Andrew Ng’s Machine Learning course, which uses Octave, you should learn Python either during the course or after since you’ll need it eventually. Additionally, another excellent Python resource is dataquest.io, which has many free Python lessons in their interactive browser environment.

[8 Best Machine Learning Software To Use in 2025 | Anaconda] July 16, 2025 - Keras is best for quickly prototyping and experimenting with neural network architectures. Getting familiar with machine learning software can seem daunting, but there are many resources that can help you get started and progress in your journey.

[Machine Learning in 2025: What learnin

# Optional (Multi-agent Deep Research)
Instead of a single multi-step agent, you can design multiple collaborating agents such as a Planner, Searcher, Summarizer, and Verifier that pass information and refine each other’s outputs. This setup improves robustness, diversity of reasoning, and division of labor.

Try building a simple setup with 2–3 agents that share goals and messages, for example Planner → Researcher → Writer.

In [ ]:
def parallel_research(query, n=3):
    # Run n independent research runs in parallel and return their answers.
    # Steps: use ThreadPoolExecutor; submit n calls to your agent/search pipeline; gather results in order.
    """
    YOUR CODE HERE
    """

answers = parallel_research("What are the best resources to learn ML in 2025?")
for i,a in enumerate(answers,1):
    print(f"[Run {i}] {a[:200]}…")

## 🎉 Congratulations!

* Practised various inference‑time reasoning methods
* Gained intuition about training reasoning models
* You have built a **deep-research agent**: reasoning model like deep-seek r1 + ReAct-style agent + tool use (web search)
* Try adding more tools, and extending the deep-research to a multi-agent system: many agents researching web in parallel.


👏 **Great job!** Take a moment to celebrate. The techniques you implemented here power many production agents and chatbots.